<a href="https://colab.research.google.com/github/baleineindigo/Data_Analysis/blob/master/%EB%AA%A8%ED%98%95%EC%B5%9C%EC%A0%81%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import validation_curve
import torch

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

## Scikit-Learn 의 모형 하이퍼 파라미터 튜닝 도구¶
Scikit-Learn에서는 다음과 같은 모형 최적화 도구를 지원한다.

- validation_curve : 단일 하이퍼 파라미터 최적화

- GridSearchCV : 그리드를 사용한 복수 하이퍼 파라미터 최적화

- ParameterGrid : 복수 파라미터 최적화용 그리드



### validation_curve 
최적화할 파라미터 이름과 범위, 그리고 성능 기준을 param_name, param_range, scoring 인수로 받아 파라미터 범위의 모든 경우에 대해 성능 기준을 계산한다.

In [ ]:
# 데이터 셋
digits=load_digits()
X,y=digits.data,digits.target

param_range=np.logspace(-6,-1,10)

# train, test 셋
%time
train_scores,test_scores=validation_curve(SVC(),X,y,
                                          param_name="gamma",param_range=param_range,
                                          cv=10,scoring="accuracy",n_jobs=1)


In [ ]:
#  평균과 표준 편차 구하기
train_scores_mean=np.mean(train_scores,axis=1)
train_scores_std=np.std(train_scores,axis=1)
test_scores_mean=np.mean(test_scores,axis=1)
test_scores_std=np.std(test_scores,axis=1)

In [ ]:
#### 시각화
plt.rcParams["font.family"] = 'DejaVu Sans'
plt.figure(figsize=(10,7))

# Training Score의 평균과 std 선
plt.semilogx(param_range,train_scores_mean,label="Training Score",color="r")
plt.fill_between(param_range,train_scores_mean-train_scores_std,
                        train_scores_mean+train_scores_std,alpha=0.2,color="r")

# Test Score의 평균과 std 선
plt.semilogx(param_range,test_scores_mean, label="Cross-validation score",color="c")
plt.fill_between(param_range,test_scores_mean-test_scores_std,
                        test_scores_mean+test_scores_std,alpha=0.2,color="c")

plt.legend(loc="best")
plt.title("Validation Curve with SVM")
plt.xlabel("$\gamma$")
plt.ylabel("Score")
plt.ylim(0.0,1.1)


### GridSearchCV 
GridSearchCV 클래스는 validation_curve 함수와 달리 모형 래퍼(Wrapper) 성격의 클래스이다. 클래스 객체에 fit 메서드를 호출하면 grid search를 사용하여 자동으로 복수개의 내부 모형을 생성하고 이를 모두 실행시켜서 최적 파라미터를 찾아준다. 생성된 복수개와 내부 모형과 실행 결과는 다음 속성에 저장된다.
\
- grid_scores_ : param_grid 의 모든 파리미터 조합에 대한 성능 결과. 각각의 원소는 다음 요소로 이루어진 튜플이다.

- parameters: 사용된 파라미터

- mean_validation_score: 교차 검증. (cross-validation) 결과의 평균값

- cv_validation_scores: 모든 교차 검증(cross-validation) 결과

- best_score_ : 최고 점수

- best_params_ : 최고 점수를 낸 파라미터

- best_estimator_ : 최고 점수를 낸 파라미터를 가진 모형


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# scl는 스케일링, slf는 모델 적용함을 의미
pipe_svc=Pipeline([('scl',StandardScaler()),('clf',SVC(random_state=1))])
# param_range로 시험해볼 파라미터 범위 정해줌
param_range=[10**i for i in range(-4,4)]
param_grid = [
    {'clf__C': param_range, 'clf__kernel': ['linear']}, # 선형SVC
    {'clf__C': param_range, 'clf__gamma': param_range, 'clf__kernel': ['rbf']} # 커널SVC
    ]

gs=GridSearchCV(estimator=pipe_svc, param_grid=param_grid,
                scoring="accuracy",cv=10,n_jobs=1)


In [ ]:
# 모델 학습 및 결과
%time 
# gs = gs.fit(X, y)
print("mean_test_score :\n ",gs.cv_results_["mean_test_score"])
print("\n best_score : ",gs.best_score_)
print("\n best_params_ : ",gs.best_params_)

#### ParameterGrid
때로는 scikit-learn 이 제공하는 GridSearchCV 이외의 방법으로 그리드 탐색을 해야하는 경우도 있다. 이 경우 파라미터를 조합하여 탐색 그리드를 생성해 주는 명령어가 ParameterGrid 이다. ParameterGrid 는 탐색을 위한 iterator 역할을 한다.

In [ ]:
from sklearn.model_selection import ParameterGrid
param_grid = {'a': [1, 2], 'b': [True, False]}
list(ParameterGrid(param_grid))

In [ ]:
param_grid = [{'kernel': ['linear']}, {'kernel': ['rbf'], 'gamma': [1, 10]}]
list(ParameterGrid(param_grid))

#### 병렬 처리
GridSearchCV 명령에는 n_jobs 라는 인수가 있다. 디폴트 값은 1인데 이 값을 증가시키면 내부적으로 멀티 프로세스를 사용하여 그리드서치를 수행한다. 만약 CPU 코어의 수가 충분하다면 n_jobs를 늘릴 수록 속도가 증가한다.
실제 하드웨어의 코어 수가 부족하다면 병렬로 실행되지 않으므로 실행시간이 단축되지 않는다.

In [ ]:
param_grid = {"gamma": np.logspace(-6, -1, 10)}
gs1 = GridSearchCV(estimator=SVC(), param_grid=param_grid,
                   scoring='accuracy', cv=5, n_jobs=1)
gs2 = GridSearchCV(estimator=SVC(), param_grid=param_grid,
                   scoring='accuracy', cv=5, n_jobs=2)



In [ ]:
%time
gs1.fit(X, y)

In [ ]:
%time
gs2.fit(X, y)